### **<span style="color:crimson">OUTER `JOIN`<span>**
* Add the data that don't have a match.
* Any value that does not match is made to be NULL

<img src="../../images/outer-join.png" height="140">

\
**`LEFT JOIN`:**

```sql
SELECT *
FROM <table A> AS a
LEFT [OUTER] JOIN <table B> AS b
ON a.id = b.id;
```

\
**`RIGHT JOIN`:**

```sql
SELECT *
FROM <table A> AS a
RIGHT [OUTER] JOIN <table B> AS b
ON a.id = b.id;
```

\
<span style="color:mediumseagreen">**Notes:**</span>
- Returns everything where `a.id` is equal to `b.id`, including non-matching rows.
    - `LEFT JOIN`s add the data that don't have a match from `table A`
    - `RIGHT JOIN`s add the data that don't have a match from `table B`
- Any value that does not match is made to be `NULL`.
___

#### **<span style="color:darkorange">OUTER `JOIN`: LEFT</span>**
 
**DATABASE: `Employees`**

**TABLE A: `employees`**

|emp_no|birth_date|first_name|last_name|gender|hire_date|
|:---:|:---:|:---:|:---:|:---:|:---:|
|10001|1953-09-02|Georgi|Facello|M|1986-06-26|
|10002|1964-06-02|Bezalel|Simmel|F|1985-11-21|
|10003|1959-12-03|Parto|Bamford|M|1986-08-28|
|10004|1954-05-01|Chirstian|Koblick|M|1986-12-01|

Number of records: 300024

\
**TABLE B: `dept_manager`**

|dept_no|emp_no|from_date|to_date|
|:---:|:---:|:---:|:---:|
|d001|110022|1985-01-01|1991-10-01|
|d001|110039|1991-10-01|9999-01-01|
|d002|110085|1985-01-01|1989-12-17|
|d002|110114|1989-12-17|9999-01-01|

Number of records: 24

\
**TABLE C: `titles`**

|emp_no|title|from_date|to_date|
|:---:|:---:|:---:|:---:|
|10001|Senior Engineer|1986-06-26|9999-01-01|
|10002|Staff|1996-08-03|9999-01-01|
|10003|Senior Engineer|1995-12-03|9999-01-01|
|10004|Engineer|1986-12-01|1995-12-01|

Number of records: 443308

#### <span style="color:hotpink">Return employee number for employees and any matching employee numbers from department managers, including non-matching "NULL" values.</span>

```sql
SELECT emp.emp_no, dep.emp_no
FROM employees AS emp
LEFT JOIN dept_manager AS dep ON emp.emp_no = dep.emp_no -- not all employees are department managers so a lot of null values in second column
```

|emp_no|emp_no|
|:---:|:---:|
|10001| |
|10002| |
|10003| |
|10004| |

Number of records: 300024

Number of fields: 2

#### <span style="color:hotpink">Filter out all of the employees that are not in the department manager column.</span>

```sql
SELECT emp.emp_no, dep.emp_no
FROM employees AS emp
LEFT JOIN dept_manager AS dep ON emp.emp_no = dep.emp_no
WHERE dep.emp_no IS NOT NULL; -- same as `INNER JOIN`
```

|emp_no|emp_no|
|:---:|:---:|
|110022|110022|
|110039|110039|
|110085|110085|
|110114|110114|

Number of records: 24

Number of fields: 2

#### <span style="color:hotpink">How many are managers?</span>

```sql
SELECT COUNT(emp.emp_no)
FROM employees AS emp
LEFT JOIN dept_manager AS dep ON emp.emp_no = dep.emp_no
WHERE dep.emp_no IS NOT NULL;
```

Answer: 24

#### <span style="color:hotpink">How many are not managers?</span>

```sql
SELECT COUNT(emp.emp_no)
FROM employees AS emp
LEFT JOIN dept_manager AS dep ON emp.emp_no = dep.emp_no
WHERE dep.emp_no IS NULL;
```

Answer: 300000


#### <span style="color:hotpink">What if we wanted to include all of the salaries that did not account for a title change?</span>

**<span style="color:mediumseagreen">Recall our example from the inner-join section, in which we wanted to display the original salary as well as the salary at promotion...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
INNER JOIN titles AS c ON c.emp_no = a.emp_no 
    AND (
        b.from_date = c.from_date
        OR (b.from_date + INTERVAL '2 days') = c.from_date
    )
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

|emp_no|salary|from_date|title|
|:---:|:---:|:---:|:---:|
|10001|60117|1986-06-26|Senior Engineer|
|10002|65828|1996-08-03|Staff|
|10003|40006|1995-12-03|Senior Engineer|
|10004|40054|1986-12-01|Engineer|

#### <span style="color:hotpink">How can we include raises that didn't result in a promotion?</span>

**<span style="color:mediumseagreen">We want all of the salaries that didn't account for a title change, so we don't want to be strict on the titles. So, lets use a `LEFT JOIN`...</span>**

```sql
SELECT a.emp_no, b.salary, b.from_date, c.title
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
LEFT JOIN titles AS c ON c.emp_no = a.emp_no 
    AND (
        b.from_date = c.from_date
        OR (b.from_date + INTERVAL '2 days') = c.from_date
    )
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

|emp_no|salary|from_date|title|
|:---:|:---:|:---:|:---:|
|10004|40054|1986-12-01|Engineer|
|10004|42283|1987-12-01| |
|10004|42542|1988-11-30| |
|10004|46065|1989-11-30| |
|10004|48271|1990-11-30| |
|10004|50594|1991-11-30| |
|10004|52119|1992-11-29| |
|10004|54693|1993-11-29| |
|10004|58326|1994-11-29| |
|10004|60770|1995-11-29|Senior Engineer|


\
**<span style="color:mediumseagreen">Notes:</span>**

- Expanding intersection to include all of the `salaries` that aren't necessarily linked to a `title` or a `from_date`.
- This is an intersection of the output table to demonstrate how we can see all of the raises, regardless of a title change.

\
**<span style="color:mediumseagreen">We can further clean up our solution...</span>**

```sql
SELECT a.emp_no, 
CONCAT(a.first_name, a.last_name) as "name",
b.salary,
COALESCE(c.title, 'No title change'),
COALESCE(c.from_date::text, '-') AS "title taken on"
FROM employees as a
INNER JOIN salaries AS b ON b.emp_no = a.emp_no
LEFT JOIN titles AS c 
ON c.emp_no = a.emp_no 
    AND (
        b.from_date = c.from_date
        OR (b.from_date + INTERVAL '2 days') = c.from_date
    )
ORDER BY a.emp_no ASC, b.from_date ASC;
```

\
**<span style="color:mediumseagreen">Result:</span>**

|emp_no|name|salary|coalesce|title taken on|
|:---:|:---:|:---:|:---:|:---:|
|10004|ChirstianKoblick|40054|Engineer|1986-12-01|
|10004|ChirstianKoblick|42283|No title change|-|
|10004|ChirstianKoblick|42542|No title change|-|
|10004|ChirstianKoblick|46065|No title change|-|
|10004|ChirstianKoblick|48271|No title change|-|
|10004|ChirstianKoblick|50594|No title change|-|
|10004|ChirstianKoblick|52119|No title change|-|
|10004|ChirstianKoblick|54693|No title change|-|
|10004|ChirstianKoblick|58326|No title change|-|
|10004|ChirstianKoblick|60770|Senior Engineer|1995-12-01|



\
**<span style="color:mediumseagreen">Notes:</span>**
- `LEFT JOIN` shows everything that is `NULL` on the left side, even if it doesn't match the criteria.
___